In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip
import glob
import re


# Replace the path below with the location where your CSV light curve files are stored

tess_lc = glob.glob("/Users/anapereira/Documents/tess_practice/CSV_FILES/tess_lc_*.csv")


for filename in tess_lc:
    # to read the csv file
    df = pd.read_csv(filename)
    
    # remove nan values
    df = df[["TIME", "PDCSAP_FLUX", "PDCSAP_FLUX_ERR"]].dropna()
    
    # Identify outliers with sigma clipping 
    s_clip = sigma_clip(df["PDCSAP_FLUX"], sigma_upper=3, sigma_lower=None)
    mask = s_clip.mask
    normal = df[~mask]
    outliers = df[mask]

    # Graph
    plt.figure(figsize=(10, 5))
    plt.errorbar(normal["TIME"], normal["PDCSAP_FLUX"], yerr=normal["PDCSAP_FLUX_ERR"],
             fmt='.', alpha=0.4, color="dimgrey")

    plt.errorbar(outliers["TIME"], outliers["PDCSAP_FLUX"], yerr=outliers["PDCSAP_FLUX_ERR"],
             fmt='.', alpha=0.7, color="palevioletred", label="Outliers")

    # Extract the number from the filename to put in plot the title 
    match = re.search(r'tess_lc_(\d+)', filename)
    name = match.group(1) if match else filename
    
    plt.title(f"TESS Light Curve {name}")
    plt.xlabel("Time - 2457000 [BJD days]")
    plt.ylabel("Flux (PDCSAP) [e-/s]")
    plt.legend()
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(filename.replace(".csv", ".png"))
    plt.show()